In [1]:

from pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn


2024-10-03 16:42:22.981223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# read the data
train_data = pd.read_csv('../Datasets/ais_train.csv', delimiter="|")
print(train_data.head())


                  time    cog   sog  rot  heading  navstat       etaRaw  \
0  2024-01-01 00:00:25  284.0   0.7    0       88        0  01-09 23:00   
1  2024-01-01 00:00:36  109.6   0.0   -6      347        1  12-29 20:00   
2  2024-01-01 00:01:45  111.0  11.0    0      112        0  01-02 09:00   
3  2024-01-01 00:03:11   96.4   0.0    0      142        1  12-31 20:00   
4  2024-01-01 00:03:51  214.0  19.7    0      215        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689  
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19  
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126  
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3  


In [33]:
# Import necessary modules
import importlib
import preprocess_filter
import feature_engineering_filter

importlib.reload(preprocess_filter)
importlib.reload(feature_engineering_filter)

from preprocess_filter import Sort_by_id, Timestamp_to_timedif, Create_sequence_vectors, Drop_diff_values, Normalizer
from feature_engineering_filter import Sequence_training_data

feature_dropper=Drop_diff_values()
normalizer = Normalizer()
id_sorter = Sort_by_id()
timestamp_to_timedif = Timestamp_to_timedif()
create_sequence_vectors = Create_sequence_vectors()
target_creator = Sequence_training_data()

#pipeline = Pipeline(filters=[timestamp_to_timedif,create_sequence_vectors,id_sorter])

#data_processed=pipeline.apply_filters(data=train_data)
#data_processed.to_csv('processed_data.csv', index=True)

In [10]:
data_cleaned=feature_dropper(train_data)
print(data_cleaned.columns)

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId'],
      dtype='object')


In [21]:
data_timestamped = timestamp_to_timedif(data_cleaned)
data_timestamped.to_csv('intermediate/data_timestamped.csv', index=True)
#data_timestamped=pd.read_csv("intermediate/data_timestamped.csv",index_col=0)
#print(data_timestamped.head())


In [26]:
data_normalized, time_scaler = normalizer(data_timestamped)
print(data_normalized.head())

        cog       sog       rot   heading   navstat  latitude  longitude  \
0  0.788889  0.006843  0.498039  0.172211  0.000000  0.108300   0.316705   
1  0.304444  0.000000  0.474510  0.679061  0.066667  0.477832   0.254259   
2  0.308333  0.107527  0.498039  0.219178  0.000000  0.734385   0.262931   
3  0.267778  0.000000  0.498039  0.277886  0.066667  0.111110   0.919778   
4  0.594444  0.192571  0.498039  0.420744  0.000000  0.706382   0.466656   

                   vesselId                time  time_diff  
0  61e9f3a8b937134a3c4bfdf7 2024-01-01 00:00:25    22123.0  
1  61e9f3d4b937134a3c4bff1f 2024-01-01 00:00:36     1980.0  
2  61e9f436b937134a3c4c0131 2024-01-01 00:01:45     1372.0  
3  61e9f3b4b937134a3c4bfe77 2024-01-01 00:03:11     3603.0  
4  61e9f41bb937134a3c4c0087 2024-01-01 00:03:51    85735.0  


In [29]:
data_sequenced=create_sequence_vectors(data_normalized)
data_sequenced.to_csv('intermediate/data_sequenced.csv', index=True)
#data_sequenced=pd.read_csv("intermediate/data_sequenced.csv",index_col=0)
#print(data_sequenced.head())

In [34]:
#create sequence training data
sequence_list, target_list = target_creator(data_sequenced,time_scaler)
#sequence_training_data.to_csv('intermediate/sequence_training_data.csv', index=True)
#sequence_training_data=pd.read_csv("intermediate/sequence_training_data.csv",index_col=0)
#print(sequence_training_data.head())
print(sequence_list.shape)
np.save('intermediate/sequence_list.npy', sequence_list)
np.save('intermediate/target_list.npy', target_list)


/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlæring/Gruppe/ML-Group-Project/src/feature_engineering_filter.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sequences = row[0]
/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlæring/Gruppe/ML-Group-Project/src/feature_engineering_filter.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sequences = row[0]
/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlæring/Gruppe/ML-Group-Project/src/feature_engineering_filter.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer ke

(1517942, 5, 7)


In [35]:
print(sequence_list.shape)

(1517942, 5, 7)


In [ ]:
from sklearn.model_selection import train_test_split

# Create train and test sets
X = sequence_training_data.drop(["vesselId", "Target"], axis=1)
Y = sequence_training_data["Target"]# create LST model

train_data, test_data, train_target, test_target = train_test_split(X, Y, test_size=0.9)

In [38]:
#Train and evaluate model


model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(5,7)),
    tf.keras.layers.LSTM(64,input_shape=(5,7)),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(2)
])   


model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(sequence_list,target_list, epochs=50, batch_size=64)
model.save("lstm_model.h5")
#loss,accuracy=model.evaluate(test_data,test_target, metrics=["mean_squared_error"])
#print("test accuracy",accuracy,"with loss",loss )

Epoch 1/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 512s 21ms/step - loss: 0.0014
Epoch 2/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 459s 19ms/step - loss: 2.3481e-04
Epoch 3/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 461s 19ms/step - loss: 1.7611e-04
Epoch 4/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 1772s 75ms/step - loss: 1.4865e-04
Epoch 5/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 499s 21ms/step - loss: 1.3419e-04
Epoch 6/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 443s 19ms/step - loss: 1.2429e-04
Epoch 7/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 409s 17ms/step - loss: 1.1720e-04
Epoch 8/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 567s 24ms/step - loss: 1.1087e-04
Epoch 9/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 689s 29ms/step - loss: 1.0591e-04
Epoch 10/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 515s 22ms/step - loss: 1.0478e-04
Epoch 11/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 540s 23ms/step - loss: 9.6834e-05
Epoch 12/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 734s 31ms/step - loss: 9.5058e-05
Epoch 13/50
23718/23718 ━━━━━━━━━━━━━━━━━━━━ 501s 21ms/step - lo